# 前期准备

In [104]:
import pandas as pd
import numpy as np
from lxml.html import fromstring
import time
import base64
import json
import requests
import os
from random import random
import requests_html
from requests_html import HTMLSession
from selenium import webdriver
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities
from PIL import Image, ImageEnhance

In [187]:
wd = webdriver.Chrome()
wd.get("https://www.baidu.com")
opts = webdriver.ChromeOptions()
opts.add_argument('--no-sandbox')#解决DevToolsActivePort文件不存在的报错
opts.add_argument('window-size=1920x3000') #指定浏览器分辨率
opts.add_argument('--disable-gpu') #谷歌文档提到需要加上一这个属性来规避bug
opts.add_argument('--hide-scrollbars') #隐藏滚动条, 应对些特殊页面
#opts.add_argument('blink-settings=imagesEnabled=false') #不加载图片, 提升速度
#opts.add_argument('--headless') #浏览器不提供可视化页面. linux下如果系统不支持可视化不加这条会启动失败
# opts.binary_location = "C:\portable\PortableApps\IronPortable\App\Iron\chrome.exe"
# opts.binary_location = "C:\Program Files\Google\Chrome\Application\chromedriver.exe" #"H:\_coding_\Gitee\InternetNewMedia\CapstonePrj2016\chromedriver.exe"  


driver = webdriver.Chrome( chrome_options = opts) #desired_capabilities=caps,

<ipython-input-187-8cdc02b6dc2f>:14: DeprecationWarning: use options instead of chrome_options
  driver = webdriver.Chrome( chrome_options = opts) #desired_capabilities=caps,


# 登录知网

In [188]:
driver.get("https://kns.cnki.net/")

In [189]:
element = driver.find_element_by_xpath('//*[@id="highSearch"]')
element.click()

In [190]:
print (driver.window_handles)

['CDwindow-3F03C2E575A876145D07EA735EC03DC3', 'CDwindow-D1EEE6774D2A661CB4B8FFED05EFD886']


In [191]:
driver.switch_to_window(driver.window_handles[1])#定位当前窗口

<ipython-input-191-15fc42e20ab9>:1: DeprecationWarning: use driver.switch_to.window instead
  driver.switch_to_window(driver.window_handles[1])#定位当前窗口


# 进行检索

In [192]:
# 点击学术期刊
element = driver.find_element_by_xpath('/html/body/div[3]/div[1]/div/ul[1]/li[1]/a')
element.click()

In [204]:
# 点击专业检索
element = driver.find_element_by_xpath('/html/body/div[2]/div/div[2]/ul/li[4]')
element.click()

In [207]:
#勾选期刊类型
element = driver.find_element_by_xpath('/html/body/div[2]/div/div[2]/div/div[1]/div[1]/div[2]/div[1]/div[3]/div/label[2]/input')
element.click()#SCI来源期刊

In [208]:
element = driver.find_element_by_xpath('/html/body/div[2]/div/div[2]/div/div[1]/div[1]/div[2]/div[1]/div[3]/div/label[4]/input')
element.click()#北大核心

In [209]:
element = driver.find_element_by_xpath('/html/body/div[2]/div/div[2]/div/div[1]/div[1]/div[2]/div[1]/div[3]/div/label[5]/input')
element.click()#CSSCI

In [210]:
element = driver.find_element_by_xpath('/html/body/div[2]/div/div[2]/div/div[1]/div[1]/div[2]/div[1]/div[3]/div/label[6]/input')
element.click()#CSCD

In [198]:
# 编辑关键词：AI_新媒体_query = '(SU="新媒体" and SU="人工智能") OR (SU="AI" and SU="新媒体")'
AI_大数据_query = '( SU="大数据" AND KY="人工智能")'

In [205]:
element = driver.find_element_by_xpath('/html/body/div[2]/div/div[2]/div/div[1]/div[1]/div[2]/textarea')
element.clear()

In [212]:
element = driver.find_element_by_xpath('/html/body/div[2]/div/div[2]/div/div[1]/div[1]/div[2]/textarea').send_keys(AI_大数据_query)#输入关键词

In [213]:
#点击检索
element = driver.find_element_by_xpath('/html/body/div[2]/div/div[2]/div/div[1]/div[1]/div[2]/div[2]/input')
element.click()

In [215]:
#选择每页50篇
element = driver.find_element_by_xpath('/html/body/div[3]/div[2]/div[2]/div[2]/form/div/div[1]/div[2]/div[2]/div/div/div/i')
element.click()

In [216]:
element = driver.find_element_by_xpath('/html/body/div[3]/div[2]/div[2]/div[2]/form/div/div[1]/div[2]/div[2]/div/div/ul/li[3]/a')
element.click()

In [124]:
# 查看页面信息
element = driver.find_element_by_id('gridTable')
page_html = element.get_attribute('innerHTML')
page_html
pd.read_html(page_html)[0]

,Unnamed: 0,篇名,作者,刊名,发表时间,被引,下载,操作
0,1,绿色理念下智慧零售创新发展路径,翟璇,商业经济研究,2021-06-30,NaN,NaN,下载
1,2,智能化金融监管：模型框架、边缘约束和实践策略 网络首发,乔宇锋,南方金融,2021-06-29 17:04,NaN,NaN,下载
2,3,“智能+”时代的现代文化产业体系:挑战与重塑 网络首发,解学芳; 雷文宣,深圳大学学报(人文社会科学版),2021-06-29 15:45,NaN,NaN,下载
3,4,管道数字孪生体模型的构建及应用,陈斯迅;李在蓉;王禹钦;关红亮;高丽,油气储运,2021-06-25,NaN,147.0,下载
4,5,大数据智能时代地图学课程内容改革探索,刘慧敏; 邓敏; 刘宝举; 陈杰,测绘通报,2021-06-25,NaN,8.0,下载
5,6,关于城市大脑未来形态的思考 网络首发,胡坚波,人民论坛·学术前沿,2021-06-18 11:09,NaN,336.0,下载
6,7,人工智能审计应用的国际进展,吴勇; 余洁; 王尚纯; 张超,中国注册会计师,2021-06-15,NaN,209.0,下载
7,8,地理空间认知理论与地图工具的发展,郑束蕾,测绘学报,2021-06-15,NaN,NaN,下载
8,9,科学数据智能:人工智能在科学发现中的机遇与挑战 网络首发,孟小峰,中国科学基金,2021-06-11 18:01,NaN,465.0,下载
9,10,测绘大数据时代数据处理理论面临的挑战与发展 网络首发,朱建军; 宋迎春; 胡俊; 邹滨; 吴立新,武汉大学学报(信息科学版),2021-06-09 10:26,NaN,259.0,下载


# 导出refworks和endnote文件

## 循环导出——endnote（分50篇各自导出）

In [125]:
# 将循环时需要点击的元素做成列表
piliang_list = []
piliang_list.append('//*[@id="gridTable"]/div[1]/div[2]/div[1]/a')
piliang_list.append("/html/body/div[3]/div[2]/div[2]/div[2]/form/div/div[1]/div[2]/div[1]/label/input")
piliang_list.append("/html/body/div[3]/div[2]/div[2]/div[2]/form/div/div[1]/div[2]/ul[1]/li[2]/i")
piliang_list.append("/html/body/div[3]/div[2]/div[2]/div[2]/form/div/div[1]/div[2]/ul[1]/li[2]/ul/li[1]/a")
piliang_list.append("/html/body/div[3]/div[2]/div[2]/div[2]/form/div/div[1]/div[2]/ul[1]/li[2]/ul/li[1]/ul/li[9]/a")
piliang_list

['//*[@id="gridTable"]/div[1]/div[2]/div[1]/a',
 '/html/body/div[3]/div[2]/div[2]/div[2]/form/div/div[1]/div[2]/div[1]/label/input',
 '/html/body/div[3]/div[2]/div[2]/div[2]/form/div/div[1]/div[2]/ul[1]/li[2]/i',
 '/html/body/div[3]/div[2]/div[2]/div[2]/form/div/div[1]/div[2]/ul[1]/li[2]/ul/li[1]/a',
 '/html/body/div[3]/div[2]/div[2]/div[2]/form/div/div[1]/div[2]/ul[1]/li[2]/ul/li[1]/ul/li[9]/a']

In [127]:
for page in range(0,26):
    for i in piliang_list:#循环点击元素的列表，依次点击
        element = driver.find_element_by_xpath(i)
        element.click()
        time.sleep(2)
    driver.switch_to.window(driver.window_handles[2])#定位新窗口
    time.sleep(10)
    driver.find_element_by_xpath('//*[@id="litotxt"]/a').click()#点击导出
    time.sleep(10)
    driver.close()#关闭新窗口
    time.sleep(4)
    driver.switch_to.window(driver.window_handles[1])#定位回原窗口
    driver.find_element_by_xpath('//*[@id="PageNext"]').click()#点击下一页
    time.sleep(12)

## 循环导出——refworks（分50篇各自导出）

In [178]:
# 将循环时需要点击的元素做成列表
piliang_list = []
piliang_list.append('//*[@id="gridTable"]/div[1]/div[2]/div[1]/a')
piliang_list.append("/html/body/div[3]/div[2]/div[2]/div[2]/form/div/div[1]/div[2]/div[1]/label/input")
piliang_list.append("/html/body/div[3]/div[2]/div[2]/div[2]/form/div/div[1]/div[2]/ul[1]/li[2]/i")
piliang_list.append("/html/body/div[3]/div[2]/div[2]/div[2]/form/div/div[1]/div[2]/ul[1]/li[2]/ul/li[1]/a")
piliang_list.append("/html/body/div[3]/div[2]/div[2]/div[2]/form/div/div[1]/div[2]/ul[1]/li[2]/ul/li[1]/ul/li[8]/a")
piliang_list

['//*[@id="gridTable"]/div[1]/div[2]/div[1]/a',
 '/html/body/div[3]/div[2]/div[2]/div[2]/form/div/div[1]/div[2]/div[1]/label/input',
 '/html/body/div[3]/div[2]/div[2]/div[2]/form/div/div[1]/div[2]/ul[1]/li[2]/i',
 '/html/body/div[3]/div[2]/div[2]/div[2]/form/div/div[1]/div[2]/ul[1]/li[2]/ul/li[1]/a',
 '/html/body/div[3]/div[2]/div[2]/div[2]/form/div/div[1]/div[2]/ul[1]/li[2]/ul/li[1]/ul/li[8]/a']

In [179]:
for page in range(0,26):
    for i in piliang_list:#循环点击元素的列表，依次点击
        element = driver.find_element_by_xpath(i)
        element.click()
        time.sleep(2)
    driver.switch_to.window(driver.window_handles[2])#定位新窗口
    time.sleep(10)
    driver.find_element_by_xpath('//*[@id="litotxt"]/a').click()#点击导出
    time.sleep(10)
    driver.close()#关闭新窗口
    time.sleep(4)
    driver.switch_to.window(driver.window_handles[1])#定位回原窗口
    driver.find_element_by_xpath('//*[@id="PageNext"]').click()#点击下一页
    time.sleep(12)
# 出现报错的原因是因为最后一页没有下一页的按键，所以在第一行代码的页面循环中只需填写到最后一页的上一页页数即可，如此次爬取的全部为27页，则填写（0,26），循环到第26页的最后一个下一页按键即可。

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"xpath","selector":"//*[@id="PageNext"]"}
  (Session info: chrome=90.0.4430.212)


# PDF原文文件下载

## 封装各功能（方便使用）

### 封装判断验证码函数

In [220]:
# 封装判断验证码
def img_have():
    try:
        driver.find_element_by_xpath('//*[@id="vImg"]')#验证码图片
        return True#为真，即有验证码图片存在
    except:
        return False#反之为空则没有验证码，返回False

### 封装截图函数

In [221]:
#封装截图
def img():
    driver.find_element_by_xpath('//*[@id="vImg"]').click()
    time.sleep(2)
    screenImg = r"D:\课程资料\大二下课程总\web数据挖掘\期末验证码\screenImg.png"#本地保存位置
    driver.get_screenshot_as_file(screenImg)
    location = driver.find_element_by_xpath('//*[@id="vImg"]').location#验证码位置
    size = driver.find_element_by_xpath('//*[@id="vImg"]').size#截图大小
    left = location['x']
    top = location['y']
    right = location['x'] + size['width']
    bottom = location['y'] + size['height']
    img = Image.open(screenImg).crop((left, top, right, bottom))
    # 优化图片
    img = img.convert('RGBA')
    img = img.convert('L')
    img = ImageEnhance.Contrast(img)
    img = img.enhance(2.0)
    img.save(screenImg)
    img = Image.open(screenImg)
    return img

### 封装图鉴API

In [222]:
# 封装图鉴API
uname = "*****"#个人隐私，不便展示，可自己注册购买使用
pwd = "*****"#个人隐私，不便展示，可自己注册购买使用
typeid  = "3"
def base64_api(uname, pwd, img, typeid):
    with open('D:\课程资料\大二下课程总\web数据挖掘\期末验证码\screenImg.png', 'rb') as f:    
        base64_data = base64.b64encode(f.read())
        img = base64_data.decode()
    data = {"username": uname, "password": pwd, "typeid": typeid, "image": img}
    time.sleep(1)
    result = json.loads(requests.post("http://api.ttshitu.com/predict", json=data).text)
    if result['success']:
        return result["data"]["result"]
    else:
        return result["message"]
    return ""

### 封装验证码输入和正确情况判断

In [223]:
#封装输入验证码
def shuru():
    yanzhengma = base64_api(uname, pwd, img, typeid)
    driver.find_element_by_xpath('//*[@id="vcode"]').clear()
    element = driver.find_element_by_xpath('//*[@id="vcode"]').send_keys(yanzhengma)

In [224]:
# 页面切换
def qiehuan():
    all_window=driver.window_handles
    new_page = len(all_window)
    driver.switch_to.window(driver.window_handles[new_page-1])

In [225]:
#封装正确情况判断
def true():
    qiehuan()
    driver.close()
    driver.switch_to.window(driver.window_handles[2])
    time.sleep(1)
    qiehuan()
    driver.close()
    driver.switch_to.window(driver.window_handles[1])
    time.sleep(1)

### 封装检查下载文件数

In [226]:
#封装检查下载文件数
def wenjian1():
    path = r'D:\课程资料\大二下课程总\web数据挖掘\期末验证码\PDF文件'#（需在selenium浏览器更改下载文件存储的地址）
    file = int(len([lists for lists in os.listdir(path) if os.path.isfile(os.path.join(path, lists))]))#通过len和int将验证码输入前的文件夹里的文件数量进行检测，获取数量为未输入验证码的文件量，需调用os模块
    return file

In [227]:
def wenjian2():
    path = r'D:\课程资料\大二下课程总\web数据挖掘\期末验证码\PDF文件'#（需在selenium浏览器更改下载文件存储的地址）
    file2 = int(len([lists for lists in os.listdir(path) if os.path.isfile(os.path.join(path, lists))]))#通过len和int将验证码输入后的文件夹里的文件数量进行检测，获取数量为输入验证码后文件夹里的文件量，需调用os模块
    return file2

In [137]:
#判断有无下载
def wenjian3():
    if file2 > file:#如果输入验证码后的文件夹里文件数量增加，也就是数量大于输入前文件数量，则代表下载成功，验证码输入正确，返回True；反之则失败，输入错误，没有下载，文件数量没有大于前一次，返回False。
        return True
    else:
        return False

## 自动循环PDF原文文件下载

In [228]:
#返回列表页 报错的时候重新运行时方便用
driver.switch_to.window(driver.window_handles[1])

In [229]:
for page in range(1,4): #先进行三页循环，可自行选择爬取页数
    for link in range(1,50):#可选择从一页中的第几篇文章开始挖取到第几篇文章结束挖取
        pdf_xpath = '//*[@id="gridTable"]/table/tbody/tr[{}]/td[2]/a'.format(link)
        driver.find_element_by_xpath(pdf_xpath).click()#点击文章进入文章详情页
        time.sleep(1)
        driver.switch_to.window(driver.window_handles[2])#定位新窗口
        driver.find_element_by_xpath('//*[@id="pdfDown"]').click()#点击下载
        time.sleep(3)
        qiehuan()#切换到最新页面
        time.sleep(4)
        img_have()#判断是否弹出验证码窗口
        abc = img_have()
        if abc is True:#如果有验证码窗口
            img()#爬取验证码
            time.sleep(2)
            base64_api(uname, pwd, img, typeid)#调用图鉴api识别验证码内容
            shuru()#填写识别出来的验证码内容
            wenjian1()#检测下载文件夹内的文件数量
            file = wenjian1()
            time.sleep(1)
            driver.find_element_by_xpath('/html/body/div/form/dl/dd/button').click()#点击验证码提交
            wenjian2()#检测点击下载后文件夹内的文件数量
            file2 = wenjian2()
            time.sleep(1)
            wenjian3()#判断下载文件夹内的文件数量是否变化
            abc2 = wenjian3()
            while abc2 == False:#如果数量没有增加，则返回False，判断等于False，则重新进行验证码爬取识别以及输入并再次检测
                img()#爬取验证码
                base64_api(uname, pwd, img, typeid)#调用图鉴api识别验证码内容
                shuru()#填写识别出来的验证码内容
                wenjian1()#检测下载文件夹内的文件数量
                file = wenjian1()
                driver.find_element_by_xpath('/html/body/div/form/dl/dd/button').click()#点击验证码提交
                wenjian2()#检测点击下载后文件夹内的文件数量
                file2 = wenjian2()
                wenjian3()#判断下载文件夹内的文件数量是否变化
                abc2 = wenjian3()
                if abc2 == True:#如果判断等于True，则代表输入正确下载成功
                    break
            true()#下载成功，情况正确，关闭窗口，返回原窗口，进行下一篇文章PDF原文文件下载   
        else:#没有验证码窗口，则直接下载原文文件，然后关闭详情页，返回文章集合页，继续下一篇PDF原文文件下载
            qiehuan()
            driver.close()
            driver.switch_to.window(driver.window_handles[1])
            time.sleep(1)
    driver.find_element_by_xpath('//*[@id="PageNext"]').click()#一页循环挖取结束，点击进入下一页
    time.sleep(2)

> - 现在我们便可以下载自己所需的所有PDF文件了。